In [1]:
!pip install transformers -q
!pip install bert-extractive-summarizer -q
!pip install sentence_transformers -q
!pip install PyPDF2 -q
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pickle
import torch
from sentence_transformers import util
import json

In [4]:
from PyPDF2 import PdfReader
theirpdfdata = PdfReader('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/multiput.pdf')
theirpdfdata = ''.join([theirpdfdata.pages[i].extract_text() for i in range(len(theirpdfdata.pages))])
theirpdfdata

' Multiclass Intent Classification for Chatbot Based \non Machine Learning Algorithm  \n \nW. M. Amir Fazamin W. Hamzah \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \namirfazamin@unisza.edu.my \nMokhairi Makhtar \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \nmokhairi@unisza.edu.my Mohd Kamir Yusof \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \nmohdkamir@unisza.edu.my \nHasnah Nawang \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \nhasnah.nawang@kberang.mrsm.edu.myIsmahafezi Ismail \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \nismahafezi@unisza.edu.my \nAzwa Abdul Aziz \nFaculty of Informatics and Computing  \nUniversiti Sultan Zainal Abidin \nTerengganu, Malaysia \nazwaaziz@gmail.com  \nAbstract — In recent years, the use of Chatbo

In [5]:
QA = torch.load('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/QA')
Sim = torch.load('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/Sim')
Sum = torch.load('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/Sum')
geomvect = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/geomvectorizer', 'rb'))
gradevect = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/gradevectorizer', 'rb'))
profilevect = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/profileectorizer', 'rb'))
geomclass = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/geomclassifier', 'rb'))
gradeclass = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/gradeclassifier', 'rb'))
profileclass = pickle.load(open('/content/drive/MyDrive/Shriram Gunasekar/Demo Model Files/profileclassifier', 'rb'))

In [6]:
def insertgeom(text):
    test_geom = geomvect.transform([text])
    return geomclass.predict(test_geom).item()

In [7]:
def insertgrade(text):
    test_grade = gradevect.transform([text])
    return gradeclass.predict(test_grade).item()

In [8]:
def insertprofile(text):
    test_profile = profilevect.transform([text])
    return profileclass.predict(test_profile).item()

In [9]:
def qanda(context,q):
    context = context
    question = q
    return QA({'context':context, 'question':question})['answer']

In [10]:
def sim(text1, text2):
    text1_representation = Sim.encode(text1)
    text2_representation = Sim.encode(text2)
    cosine_sim = util.pytorch_cos_sim(text1_representation,text2_representation)
    return round(cosine_sim.item(),3)

In [11]:
def summ(text, numsent):
    return Sum(text, num_sentences=int(numsent))

In [12]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("Use The Different Tabs")

    with gr.Tab(" Q & A "):
        qa_data = gr.Textbox(label="Enter Data")
        qa_input = gr.Textbox(label="Enter Question")
        qa_output = gr.Textbox(label="Your Answer")
        qa_button = gr.Button('Get My Answer')

    with gr.Tab(" Summarizer "):
        sum_data = gr.Textbox(label="Enter Data")
        sum_input = gr.Textbox(label="Enter Number Of Sentences In Summary")
        sum_output = gr.Textbox(label="Your Summary")
        sum_button = gr.Button('Get My Summary')

    with gr.Tab(" Similarity "):
        sim_input1 = gr.Textbox(label="Enter First Data")
        sim_input2 = gr.Textbox(label="Enter Second Data")
        sim_output = gr.Textbox(label="Your Similarity Score")
        sim_button = gr.Button('Get Similarity Score')

    with gr.Tab("Insert Grade"):
        ig_input = gr.Textbox(label="Enter Insert Grade Description")
        ig_output = gr.Textbox(label="Recommended Insert Grade")
        ig_button = gr.Button('Get Insert Grade Recommendation')

    with gr.Tab("Insert Geometry"):
        ige_input = gr.Textbox(label="Enter Insert Geometry Description")
        ige_output = gr.Textbox(label="Recommended Insert Geometry")
        ige_button = gr.Button('Get Insert Geometry Recommendation')

    with gr.Tab("Insert Profile"):
        ip_input = gr.Textbox(label="Enter Insert Profile Description")
        ip_output = gr.Textbox(label="Your Recommended Insert Geometry Description")
        ip_button = gr.Button(label='Get Insert Profile Recommendation')


    qa_button.click(qanda, inputs=[qa_data, qa_input], outputs=qa_output)
    sim_button.click(sim, inputs=[sim_input1, sim_input2], outputs=sim_output)
    sum_button.click(summ, inputs=[sum_data, sum_input], outputs=sum_output)
    ig_button.click(insertgrade, inputs=[ig_input], outputs=ig_output)
    ige_button.click(insertgeom, inputs=[ige_input], outputs=ige_output)
    ip_button.click(insertprofile, inputs=[ip_input], outputs=ip_output)

demo.launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d701051363952d5a6c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
